<a href="https://colab.research.google.com/github/MicaelaRomeroC3/COPs/blob/main/06_Tablacategorias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [25]:
import pandas as pd
import os

In [26]:
ruta_historical ='/content/drive/MyDrive/COPS/DATA/3.HISTORICAL/llamadas.parquet'
ruta_salida = '/content/drive/MyDrive/COPS/OUTPUT/TABLA_CATEGORIAS.xlsx'
os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)

# === Cargar histórico ===
if os.path.exists(ruta_historical):
    df = pd.read_parquet(ruta_historical)
    print(f"📊 Histórico cargado con {len(df)} registros.")
else:
    print("⚠️ No se encontró el archivo histórico.")
    exit()

📊 Histórico cargado con 10000 registros.


In [27]:
# === Variables de cada tipo ===
columnas_eval_1 = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'S1', 'S2', 'S3', 'S4', 'S5M', 'C1M', 'C2M', 'T1M', 'T2M', 'T3']
ponderacion_dict = {
    'P1': 0.02, 'P2': 0.10, 'P3': 0.03, 'P4': 0.03, 'P5': 0.05, 'P6': 0.02,
    'S1': 0.03, 'S2': 0.02, 'S3': 0.10, 'S4': 0.10, 'S5M': 0.05,
    'C1M': 0.10, 'C2M': 0.10, 'T1M': 0.05, 'T2M': 0.10, 'T3': 0.10
}

# === Lista de dimensiones ===
dimensiones = ['PRODUCTO', 'SEGMENTO', 'R_NR', 'PERIMETRO']
tablas_por_dimension = []

# === Función para generar análisis por dimensión ===
for dim in dimensiones:
    errores = df.groupby(['FECHA_EVALUACIÓN', dim])[columnas_eval_1].apply(lambda x: (x == False).sum()).reset_index()
    llamadas = df.groupby(['FECHA_EVALUACIÓN', dim]).size().reset_index(name='NUM_LLAMADAS')
    resultado = pd.merge(errores, llamadas, on=['FECHA_EVALUACIÓN', dim])

    resultado = resultado.melt(
        id_vars=['FECHA_EVALUACIÓN', dim, 'NUM_LLAMADAS'],
        value_vars=columnas_eval_1,
        var_name='VARIABLE',
        value_name='N_FALSOS'
    )

    resultado['%_CUMPLIMIENTO'] = ((1 - resultado['N_FALSOS'] / resultado['NUM_LLAMADAS']) * 100).round(2)
    resultado['PONDERACION'] = resultado['VARIABLE'].map(ponderacion_dict)

    resultado['DIMENSIÓN'] = dim.capitalize()
    resultado['VALOR_DIMENSIÓN'] = resultado[dim]
    resultado.drop(columns=[dim], inplace=True)

    tablas_por_dimension.append(resultado)

# === Concatenar todas las tablas por dimensión en una tabla general ===
tabla_general = pd.concat(tablas_por_dimension, ignore_index=True)

# === Cálculo %_CUMPLIMIENTO_P, C, S, T ===
def calcular_cumplimiento_grupo(df, grupo, divisor):
    variables_grupo = {
        'P': ['P1', 'P2', 'P3', 'P4', 'P5', 'P6'],
        'S': ['S1', 'S2', 'S3', 'S4', 'S5M'],
        'C': ['C1M', 'C2M'],
        'T': ['T1M', 'T2M', 'T3']
    }
    temp = df[df['VARIABLE'].isin(variables_grupo[grupo])].copy()
    temp['PON_CUMP'] = temp['%_CUMPLIMIENTO'] * temp['PONDERACION']
    resultado = (
        temp.groupby(['VALOR_DIMENSIÓN', 'FECHA_EVALUACIÓN'])['PON_CUMP'].sum().reset_index()
    )
    resultado[f'%_CUMPLIMIENTO_{grupo}'] = (resultado['PON_CUMP'] / divisor).round(2)
    resultado.drop(columns='PON_CUMP', inplace=True)
    return resultado

cumplimiento_P = calcular_cumplimiento_grupo(tabla_general, 'P', 0.25)
cumplimiento_S = calcular_cumplimiento_grupo(tabla_general, 'S', 0.30)
cumplimiento_C = calcular_cumplimiento_grupo(tabla_general, 'C', 0.20)
cumplimiento_T = calcular_cumplimiento_grupo(tabla_general, 'T', 0.25)

# === Añadir a tabla_general ===
for df_cumpl in [cumplimiento_P, cumplimiento_S, cumplimiento_C, cumplimiento_T]:
    tabla_general = pd.merge(tabla_general, df_cumpl, on=['VALOR_DIMENSIÓN', 'FECHA_EVALUACIÓN'], how='left')

# === Añadir %_CUMPLIMIENTO_G (global) ===
def calcular_cumplimiento_global(df):
    temp = df[df['VARIABLE'].isin(columnas_eval_1)].copy()
    temp['PON_CUMP'] = temp['PONDERACION'] * temp['%_CUMPLIMIENTO']
    resultado = (
        temp.groupby(['VALOR_DIMENSIÓN', 'FECHA_EVALUACIÓN'])['PON_CUMP'].sum()
        .reset_index()
        .rename(columns={'PON_CUMP': '%_CUMPLIMIENTO_G'})
    )
    return resultado

cumplimiento_global = calcular_cumplimiento_global(tabla_general)
tabla_general = pd.merge(tabla_general, cumplimiento_global, on=['VALOR_DIMENSIÓN', 'FECHA_EVALUACIÓN'], how='left')


In [28]:
# === Tablas para hojas resumen globales ===
columnas_eval_1 = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6',
                   'S1', 'S2', 'S3', 'S4', 'S5M',
                   'C1M', 'C2M',
                   'T1M', 'T2M', 'T3']

ponderacion_dict = {
    'P1': 0.02, 'P2': 0.10, 'P3': 0.03, 'P4': 0.03, 'P5': 0.05, 'P6': 0.02,
    'S1': 0.03, 'S2': 0.02, 'S3': 0.10, 'S4': 0.10, 'S5M': 0.05,
    'C1M': 0.10, 'C2M': 0.10,
    'T1M': 0.05, 'T2M': 0.10, 'T3': 0.10
}

dimensiones_resumen = ['PRODUCTO', 'SEGMENTO', 'R_NR', 'PERIMETRO']
resumen_globales = {}

for dim in dimensiones_resumen:
    # Empezamos desde cero, sobre df crudo
    errores = df.groupby(dim)[columnas_eval_1].apply(lambda x: (x == False).sum()).reset_index()
    llamadas = df.groupby(dim).size().reset_index(name='NUM_LLAMADAS')
    temp = pd.merge(errores, llamadas, on=dim)

    temp = temp.melt(
        id_vars=[dim, 'NUM_LLAMADAS'],
        value_vars=columnas_eval_1,
        var_name='VARIABLE',
        value_name='N_FALSOS'
    )

    temp['%_CUMPLIMIENTO'] = ((1 - temp['N_FALSOS'] / temp['NUM_LLAMADAS']) * 100).round(2)
    temp['PONDERACION'] = temp['VARIABLE'].map(ponderacion_dict)
    temp['PON_CUMP'] = (temp['%_CUMPLIMIENTO'] / 100) * temp['PONDERACION']

    resumen = temp.groupby(dim, as_index=False)['PON_CUMP'].sum()
    resumen['%_CUMPLIMIENTO_GLOBAL'] = (resumen['PON_CUMP'] * 100).round(2)
    resumen_globales[dim] = resumen[[dim, '%_CUMPLIMIENTO_GLOBAL']]


print(resumen_globales[dim])

  PERIMETRO  %_CUMPLIMIENTO_GLOBAL
0      0-30                  99.47
1   180-365                  99.06
2     30-90                  99.66
3      365m                  99.72
4    90-180                  99.33


In [29]:
# === Exportar a Excel ===
with pd.ExcelWriter(ruta_salida, engine='openpyxl', mode='w') as writer:

    # 1️⃣ Guardar hojas detalladas por dimensión (código viejo)
    for dim in dimensiones:
        df_dim = tabla_general[tabla_general['DIMENSIÓN'] == dim.capitalize()]
        df_dim.to_excel(writer, sheet_name=dim.capitalize(), index=False)

    # 2️⃣ Guardar tabla general concatenada
    tabla_general.to_excel(writer, sheet_name='TABLA_GENERALCATEG', index=False)

    # 3️⃣ Guardar resultados globales calculados desde cero
    for dim, df_resumen in resumen_globales.items():
        df_resumen.to_excel(writer, sheet_name=f'{dim.capitalize()}_GLOBAL', index=False)

print(f"✅ Excel generado correctamente en: {ruta_salida}")


✅ Excel generado correctamente en: /content/drive/MyDrive/COPS/OUTPUT/TABLA_CATEGORIAS.xlsx
